### Imports

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import re
import os
import time
from random import randint

In [2]:
os.environ['PATH'] = r"/Users/aniekan/seleniumDrivers/bin/"
browser = webdriver.Chrome()

In [3]:
typ = []
address = []
desc = []
beds = []
baths = []
price = []
agent = []
tel = []

In [4]:
pages = np.arange(1, 401)

s = time.time()
for page in pages:
    browser.get('https://www.zoopla.co.uk/to-rent/property/london/?page_size=25&price_frequency=per_month&q=London&radius=0&results_sort=newest_listings&search_source=refine&pn='+ str(page))
    content = browser.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    listings = soup.find('div', class_='css-1anhqz4-ListingsContainer earci3d2')

    time.sleep(randint(2,10))
    
    for listing in listings:
        try:
            phone = listing.find('a', attrs={'data-testid':'agent-phone-number'}).get_text()
            tel.append(phone)
        except: tel.append('')
            
        try:
            addy = listing.find(attrs={'data-testid':'listing-description'}).get_text()
            address.append(addy)
        except: address.append('')
        
        try:
            d = listing.h2.text
            desc.append(d)
            try:
                t = re.search(r'(\d+\sbed)?(.*?) to rent', d).group(2).strip().capitalize()
                typ.append(t)
            except: typ.append('')
        except: desc.append('')
            
        try: price.append(listing.p.text)
        except: price.append('')
        
        try: 
            ag = listing.find('img', class_='lazy')['alt']
            agent.append(ag)
        except: agent.append('')
        
        try:
            spec = list(listing.find(attrs={'data-testid':'listing-spec'}).children)
            bdc = 0
            bthc = 0
            for rm in spec:
                if rm.span.span['data-testid'] == 'bed':
                    beds.append(rm.text)
                    bdc +=1
                if rm.span.span['data-testid'] == 'bath':
                    baths.append(rm.text)
                    bthc += 1
            if bdc == 0:
                beds.append('')
            if bthc == 0:
                baths.append('')
        except:
            beds.append('')
            baths.append('')

st = time.time()
print(f'time taken: {st-s:.5f}s')


time taken: 3695.71298s


In [14]:
mins, sec = divmod(st-s, 60)
hrs, mins = divmod(mins, 60)
print(f'HH:{int(hrs)} MM:{int(mins)} SS:{sec:.5f}') 

HH:1 MM:1 SS:35.71298


In [7]:
table = pd.DataFrame({"PropertyType":typ, "Address":address, "Description":desc, "Bedrooms":beds, "Bathrooms":baths, 
                      "Price (£)":price, "Agent":agent, "Tel":tel})
table

,PropertyType,Address,Description,Bedrooms,Bathrooms,Price (£),Agent,Tel
0,Flat,"Carrara Tower, 1 Bollinder Place, London EC1V",2 bed flat to rent,2,2,"£4,550 pcm","Marketed by London Capital Homes Limited, SW1V",020 8166 4067
1,Flat,"Carrara Tower, 1 Bollinder Place, London EC1V",2 bed flat to rent,2,2,"£5,547 pcm","Marketed by London Capital Homes Limited, SW1V",020 8166 4067
2,Flat,"Portugal Street, London WC2A",1 bed flat to rent,1,1,"£2,817 pcm","Marketed by OpenRent, EC1N",020 3542 2178
3,Flat,"Elmers End Road, London SE20",1 bed flat to rent,1,1,£900 pcm,"Marketed by Purplebricks, Head Office, B90",024 7511 8873
4,Flat,"Callow Street, London SW3",1 bed flat to rent,1,1,"£3,680 pcm","Marketed by Homelike Internet GmbH, W12",020 8115 2771
...,...,...,...,...,...,...,...,...
9995,Flat,"South Park Road, London SW19",1 bed flat to rent,1,1,"£1,300 pcm","Marketed by Hawes & Co - Wimbledon Village, SW19",020 3641 6210
9996,Studio,"Hermitage Road, London N4",Studio to rent,1,1,"£1,000 pcm","Marketed by Theodore & Charles Ltd, N7",020 8033 9219
9997,Flat,"Niton Street, Bishops Park SW6",3 bed flat to rent,3,2,"£3,250 pcm","Marketed by Carter Jonas - Parsons Green, Fulh...",020 8033 3976
9998,Studio,"Archway Road, Highgate N6",Studio to rent,,1,"£1,200 pcm","Marketed by PropertyTime, NW3",020 3478 2900


In [166]:
t = table[table != '']
t.isnull().sum()

PropertyType       0
Address            0
Description        0
Bedrooms        1015
Bathrooms        891
Price (£)          0
Agent             22
Tel                0
dtype: int64

In [310]:
ind = t.loc[t.Agent.isnull()]
ind

,PropertyType,Address,Description,Bedrooms,Bathrooms,Price (£),Agent,Tel
176,Flat,"Shaftesbury Avenue, Covent Garden WC2H",1 bed flat to rent,1,1,"Marketed by Miki Property, NW5",NaN,020 3641 2372
230,Studio,"High Road, Neasden NW10",Studio to rent,NaN,1,"Marketed by Miki Property, NW5",NaN,020 3641 2372
485,Flat,"Western Road, Romford RM1",1 bed flat to rent,1,1,"Marketed by Haverstock Property Services, RM2",NaN,01708 954867
1543,Studio,"High Road, Neasden NW10",Studio to rent,NaN,1,"Marketed by Miki Property, NW5",NaN,020 3641 2372
1544,Flat,"Bryantwood Road, Drayton Park N7",1 bed flat to rent,1,1,"Marketed by Miki Property, NW5",NaN,020 3641 2372
3652,Flat,"Finchley Road, Swiss Cottage NW3",2 bed flat to rent,2,1,"Marketed by V.M.T. Estates Limited, N17",NaN,020 8115 5403
4140,Room,"Wells Road, London NW10",Room to rent,1,1,"Marketed by City Properties, CM14",NaN,020 8022 9051
4154,Room,"Wells House Road, London NW10",Room to rent,1,1,"Marketed by City Properties, CM14",NaN,020 8022 9051
5300,Studio,"Vanston Place, Fulham SW6",Studio to rent,NaN,1,"Marketed by Ewing Property Consultant, W10",NaN,020 3641 9140
6460,Flat,"Shaftesbury Avenue, Covent Garden WC2H",1 bed flat to rent,1,1,"Marketed by Miki Property, NW5",NaN,020 3641 2372


In [311]:
ind = ind.index.to_list()

missing = ['£2348 pcm', '£875 pcm', '£900 pcm', '£854 pcm', '£1400 pcm', '£2383 pcm', '£700 pcm', '£700 pcm', 
           '£950 pcm','£2349 pcm', '£800 pcm', '£1300 pcm', '£2000 pcm', '£700 pcm', '£500 pcm', '£500 pcm', 
           '£450 pcm', '£1195 pcm', '£650 pcm', '£800 pcm', '£1200 pcm', '£900 pcm']


In [313]:
t['Missed'] = np.nan
for i, j in enumerate(ind):
    t.loc[j, 'Missed'] = missing[i]
    
def fill_missing(row):
    if row.PropertyType == 'Studio':
        row.Bedrooms = 1
        if pd.isnull(row.Bathrooms):
            row.Bathrooms = 1 
        
    if pd.isnull(row.Agent):
        row.Agent = row['Price (£)']
        row['Price (£)'] = row.Missed
    return row

t.apply(fill_missing, axis='columns')
t = t.drop(columns=['Missed'])

In [314]:
t.isna().sum()

PropertyType      0
Address           0
Description       0
Bedrooms        135
Bathrooms       774
Price (£)         0
Agent             0
Tel               0
dtype: int64

In [315]:
t.iloc[ind, :]

,PropertyType,Address,Description,Bedrooms,Bathrooms,Price (£),Agent,Tel
176,Flat,"Shaftesbury Avenue, Covent Garden WC2H",1 bed flat to rent,1,1,£2348 pcm,"Marketed by Miki Property, NW5",020 3641 2372
230,Studio,"High Road, Neasden NW10",Studio to rent,1,1,£875 pcm,"Marketed by Miki Property, NW5",020 3641 2372
485,Flat,"Western Road, Romford RM1",1 bed flat to rent,1,1,£900 pcm,"Marketed by Haverstock Property Services, RM2",01708 954867
1543,Studio,"High Road, Neasden NW10",Studio to rent,1,1,£854 pcm,"Marketed by Miki Property, NW5",020 3641 2372
1544,Flat,"Bryantwood Road, Drayton Park N7",1 bed flat to rent,1,1,£1400 pcm,"Marketed by Miki Property, NW5",020 3641 2372
3652,Flat,"Finchley Road, Swiss Cottage NW3",2 bed flat to rent,2,1,£2383 pcm,"Marketed by V.M.T. Estates Limited, N17",020 8115 5403
4140,Room,"Wells Road, London NW10",Room to rent,1,1,£700 pcm,"Marketed by City Properties, CM14",020 8022 9051
4154,Room,"Wells House Road, London NW10",Room to rent,1,1,£700 pcm,"Marketed by City Properties, CM14",020 8022 9051
5300,Studio,"Vanston Place, Fulham SW6",Studio to rent,1,1,£950 pcm,"Marketed by Ewing Property Consultant, W10",020 3641 9140
6460,Flat,"Shaftesbury Avenue, Covent Garden WC2H",1 bed flat to rent,1,1,£2349 pcm,"Marketed by Miki Property, NW5",020 3641 2372


In [316]:
table.to_excel('rent_properties_London.xlsx', index=False)